### Installation

In [1]:
%%capture
# Skip restarting message in Colab
#import sys; modules = list(sys.modules.keys())
#for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

In [3]:
!nvidia-smi

Sat Apr 12 14:12:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77                 Driver Version: 565.77         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               On  |   00000000:8A:00.0 Off |                  Off |
|  0%   25C    P8             17W /  230W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
os.environ["VLLM_USE_V1"] = '0'

In [ ]:
!pip install unsloth 
!pip install vllm==0.8.2
!pip install --upgrade pillow

In [4]:
!pip install bitsandbytes==0.45.5


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


### Loading the model

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [7]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Load up `Llama 3.1 8B Instruct`, and set parameters

In [8]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
saving_name= "moslehGen/"+model_name+"-500Steps"

In [9]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.2. vLLM: 0.8.2.
   \\   /|    NVIDIA RTX A5000. Num GPUs = 1. Max memory: 23.573 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/deepseek-r1-distill-llama-8b-unsloth-bnb-4bit with actual GPU utilization = 59.38%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.57 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.82 GB. Also swap space = 6 GB.
INFO 04-13 10:24:57 [config.py:585] This model supports multiple tasks: {'embed', 'reward', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbyt

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-13 10:25:16 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-13 10:25:17 [model_runner.py:1146] Model loading took 5.7737 GB and 15.958262 seconds
INFO 04-13 10:25:23 [worker.py:267] Memory profiling takes 5.74 seconds
INFO 04-13 10:25:23 [worker.py:267] the current vLLM instance can use total_gpu_memory (23.57GiB) x gpu_memory_utilization (0.59) = 14.00GiB
INFO 04-13 10:25:23 [worker.py:267] model weights take 5.77GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.89GiB; the rest of the memory reserved for KV Cache is 7.28GiB.
INFO 04-13 10:25:23 [executor_base.py:111] # cuda blocks: 3728, # CPU blocks: 3072
INFO 04-13 10:25:23 [executor_base.py:116] Maximum concurrency for 512 tokens per request: 116.50x
INFO 04-13 10:25:31 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If 

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:34<00:00,  1.26s/it]

INFO 04-13 10:26:05 [model_runner.py:1570] Graph capturing finished in 34 secs, took 0.61 GiB
INFO 04-13 10:26:05 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 48.66 seconds



Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
<a name="Data"></a>



In [10]:
dataPath = "converted_data.json"

In [11]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    #data = load_dataset("openai/gsm8k", "main", split=split).select(range(1))
    data = load_dataset("json", data_files=dataPath, split="train")
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['Question']}
        ],
        'answer': extract_hash_answer(x['Response'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/25263 [00:00<?, ? examples/s]

In [12]:


dataset.remove_columns(['Complex_CoT','answer'])

Dataset({
    features: ['Question', 'Response', 'prompt'],
    num_rows: 25263
})

In [13]:
dataset["Response"][0]

'"Der Skipass ist jetzt sooo teuer."'

## Define Reward Function
We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [15]:
# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]


def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

<a name="Train"></a>
## Train the model

Now set up GRPO Trainer and all configurations!

In [18]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 500,
    save_steps = 100,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)


Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,263 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)


-------------------- Question:
Wie bewerten irische Urlauber im Alter von 60 Jahren den Zustand der Gastronomie während ihres Österreich-Urlaubs in der Vorsaison? 
Answer:
None 
Response:
Zunächst betrachten wir die alterntative Aufgabenstellung: Wie bewerten österreichische Urlauber im Alter von 49 Jahren den Zugang zu Wander- und Mountainbike-Karten in Tourismusverbänden?

1. **Allgemeinbildung und Medienverbrauch**:
   - 49 Jahre alte österreichische Urlauber sind meist erfahren undwereinigt, besitzen vielleicht ein relationalisches Wissen oder suchen bereits bewusst Information zusätzlicher Quellen.
   - Print- und Onlinesuche nach Informationen über Wander- und Mountainbike-Touren ist boomend, insbesondere durch die zunehmende Digitalisierung.

2. **Technologische Nutzung**:
   - Viele Urlauber verwenden Smartphones und Tablets, um Informationen online zu recherchieren.
   - Apps und Onlineportale sind wichtiger Bestandteil der Informationssuche.
   - Die Verfügbarkeit der Touren 

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,-0.036500,0.158725,200.000000,0.000000,-0.036500,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,200.000000,0.000875,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,200.000000,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.020833,0.051031,200.000000,0.001092,0.020833,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,200.000000,0.001198,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.020833,0.051031,200.000000,0.000806,0.020833,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,200.000000,0.000724,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,200.000000,0.000891,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,200.000000,0.000829,0.000000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Wie bewerten österreichische Urlauber im Alter von 71 Jahren den allgemeinen Aufenthalt in Österreich? 
Answer:
None 
Response:
Okay, ich muss die Bewertung osteuropäischer Urlaubsgäste im Alter von 71 Jahren in Österreich ermitteln. Dafür brauche ich wahrscheinlich verschiedene Aspekte zu berückschättern, insbesondere wie diese Gäste den Aufenthalt in Österreich empfinden. Zunächst denke ich über die Aufenthaltsgestaltung ab – wie ist die Landschaft und die Städte? Österreich ist bekannt für seine Gebirge, Alpen, Weinberge und historische Städte wie Wien. Vielleichtisten Gäste finden die Aussichtsplätze, Wanderwege und kulturellen Besonderheiten interessant.

Dann überlege ich die infrastrukturellen Aspekte: gut_topic/Wienerinhalte: Bahn, Bus, Straßenwerke – stellen Unterscheidende Aspekte dar bereit, wieemütterliche Hinwegung, Sageng 
Extracted:
Okay, ich muss die Bewertung osteuropäischer Urlaubsgäste im Alter von 71 Jahren in Österreich ermitteln. Daf

<a name="Inference"></a>
## Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)

output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [28]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:27<00:00, 27.54s/it, est. speed input: 1.16 toks/s, output: 17.47 toks/s]


"Okay, so I need to figure out how to calculate pi, which is approximately 3.14159... but I don't know the exact method right now. I remember something about polygons and areas. Let me think. Oh, right, Archimedes used a method with polygons to approximate pi. He used a polygon with a lot of sides to get a better approximation.\n\nSo, how does that work? I think it's something to do with the area of a circle and a polygon. If I have a regular polygon inscribed in a circle, the area can be compared to the area of the circle to find pi. Hmm, the formula for the area of a regular polygon is something like (1/2) * perimeter * apothem. But wait, how does that relate to the circle's area?\n\nWait, the area of the circle is πr², right? And the area of the polygon is (1/2) * perimeter * apothem. If I make the polygon have many sides, the polygon's area should get closer to the circle's area. So, maybe I can set up an equation where the area of the polygon is less than the area of the circle, a

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
## Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if True: model.save_pretrained_merged(saving_name, tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged(saving_name, tokenizer, save_method = "merged_16bit", token = "YOUR_HF_TOKEN",)
        #model.push_to_hub_merged(saving_name, tokenizer, save_method = "lora", token = "")
# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if True: model.save_pretrained_merged(saving_name, tokenizer, save_method = "lora",)
if True: model.push_to_hub_merged(saving_name, tokenizer, save_method = "lora", token = "YOUR_HF_TOKEN")

In [ ]:
if True: model.save_pretrained_merged(saving_name, tokenizer, save_method = "lora",)
if True: model.push_to_hub_merged(saving_name, tokenizer, save_method = "lora", token = "YOUR_HF_TOKEN")

## GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.synchronize()


In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the base model from Hugging Face (you don't have to save it yourself)
base_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B", device_map="auto")

# Load your adapter on top
model = PeftModel.from_pretrained(base_model, "moslehGen/deepseek-llama3.1")

# Tokenizer can also be loaded from your adapter repo or the base model
tokenizer = AutoTokenizer.from_pretrained("moslehGen/deepseek-llama3.1")

# Inference
inputs = tokenizer("Your prompt here", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)